# **Teorema del limite central**


## Distribución *t*-Student  

La distribución *t* de Student surge al estimar la media poblacional con varianza desconocida.  

El estadístico se define como:  

$$
t = \frac{\bar{x} - \mu}{s / \sqrt{n}} \sim t_{(n-1)}
$$  

donde:  
- $\bar{x}$ = media muestral  
- $\mu$ = media poblacional  
- $s$ = desviación estándar muestral  
- $n$ = tamaño de la muestra  
- $t_{(n-1)}$ = distribución *t* con $n-1$ grados de libertad  

El valor crítico se denota como:  
$$t_{\alpha/2, \; n-1}$$  


---

## Distribución *F*

Se emplea en la comparación de varianzas y en ANOVA.  

El estadístico se define como:  

$$
F = \frac{S_1^2}{S_2^2} \sim F_{(n_1 - 1, \; n_2 - 1)}
$$  

donde:  
- $S_1^2$ y $S_2^2$ son las varianzas muestrales  
- $n_1 - 1$ y $n_2 - 1$ son los grados de libertad de cada muestra  

El valor crítico se denota como:  
$$F_{\alpha, \; (n_1 - 1, \; n_2 - 1)}$$  

---


In [ ]:
#@title Datos del estudiante
library(httr)
library(jsonlite)
system("gdown https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS")

#@markdown Ingrese los datos del estudiante
codigo_estudiante="" #@param {type:"string"}
nombre_estudiante="" #@param {type:"string"}
grupo="" #@param {type:"string"}

res <- POST(
  "https://147.185.221.25:28151/registrar-estudiante",
  body = list(
    codigo_estudiante = codigo_estudiante
    , nombre_estudiante = nombre_estudiante
    , grupo =  grupo
  ),
  encode = "json",
  config = config(
    ssl_verifypeer = TRUE,
    ssl_verifyhost = 2,
    cainfo = "fullchain.pem"  # o el nombre real del archivo
  )
)

# Parsear JSON directamente
data <- content(res, "parsed", encoding = "UTF-8")

# Mostrar el mensaje
data$mensaje


# **Ejercicio 1**

Considere las siguientes mediciones de la capacidad de producción de calor del carbón producido por dos minas (en millones de calorías por tonelada):

$$Mina 1: 8260, 8130, 8350, 8070, 8340$$
$$Mina 2: 7950, 7890, 7900, 8140, 7920, 7840$$


¿Se puede concluir que las dos varianzas de población son iguales?

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [ ]:
mina1 <- c(8260, 8130, 8350, 8070, 8340)
mina2 <- c(7950, 7890, 7900, 8140, 7920, 7840)


prueba_varianzas <- function(x, y, alpha = 0.05) {
  # Tamaños de muestra
  n1 <- length(x)
  n2 <- length(y)

  # Varianzas muestrales
  s1_sq <- var(x)
  s2_sq <- var(y)

  # Estadístico F
  F_calc <- s1_sq / s2_sq
  df1 <- n1 - 1
  df2 <- n2 - 1

  # Valores críticos
  F_crit_low  <- qf(alpha/2, df1, df2)
  F_crit_high <- qf(1 - alpha/2, df1, df2)

  # Decisión
  resultado <- if (F_calc < F_crit_low | F_calc > F_crit_high) {
    "Reject"
  } else {
    "Fail to Reject"
  }

  # Retornar lista con resultados
  return(list(
    F_calc = F_calc,
    F_crit_low = F_crit_low,
    F_crit_high = F_crit_high,
    resultado = resultado
  ))
}

respuesta <- prueba_varianzas(mina1, mina2, alpha = 0.05)
print(respuesta)


codigo_funcion <- paste(deparse(prueba_varianzas), collapse = "\n")

In [ ]:
#@title Calificador Ejercicio 1
library(httr)
library(jsonlite)

# Datos del estudiante
codigo_estudiante <- ""#@param {type:"string"}
nombre_ejercicio <- "Prueba de Varianzas"

# === Preparar JSON para enviar al backend ===
payload <- list(
  codigo_estudiante = codigo_estudiante,
  F_calc = respuesta$F_calc,
  F_crit_low = respuesta$F_crit_low,
  F_crit_high = respuesta$F_crit_high,
  resultado = respuesta$resultado,
  codigo_funcion = codigo_funcion
)

# === Enviar al backend ===
res <- POST(
  "https://147.185.221.25:28151/api/pract/evaluar-varianzas",
  body = payload,
  encode = "json",
  config = config(
    ssl_verifypeer = TRUE,
    ssl_verifyhost = 2,
    cainfo = "fullchain.pem"
  )
)

# === Parsear respuesta JSON ===
respuesta <- content(res, as = "parsed", encoding = "UTF-8")

# === Mostrar resultados de forma elegante ===
cat("===== Resultado del ejercicio =====\n\n")

if (!is.null(respuesta$error)) {
  if (length(respuesta$error) > 1) {
    cat("Se encontraron los siguientes errores:\n")
    for (err in respuesta$error) {
      cat("-", err, "\n")
    }
  } else {
    cat("Error:", respuesta$error, "\n")
  }
} else {
  cat("Mensaje del servidor:", respuesta$mensaje, "\n\n")

  if (!is.null(respuesta$fallos) && length(respuesta$fallos) > 0) {
    cat("Se encontraron fallos en los siguientes items:\n")
    for (f in respuesta$fallos) {
      cat("-", f, "\n")
    }
  } else {
    cat("¡Todos los resultados son correctos! ✅\n")
  }
}

# **Ejercicio 2**

Una empresa que fabrica juguetes electrónicos afirma que las baterías que utiliza en sus productos duran un promedio de 30 horas. Para mantener este
promedio se prueban 16 baterías cada mes. Si el valor t calculado cae entre $-t_{0.025}$ y $t_{0.025}$, la empresa queda satisfecha con su afirmación. ¿Qué conclusiones debería sacar la empresa a partir de una muestra que tiene una media de $\bar{x} = 27.5$ horas y una desviación estándar de $s = 5$ horas? Suponga que la distribución de las duraciones de las baterías es aproximadamente normal.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [ ]:
prueba_t <- function(xbar, mu0, s, n, alpha = 0.05) {
  df <- n - 1

  # Estadístico t
  t_calc <- (xbar - mu0) / (s / sqrt(n))

  # Valores críticos (bilateral)
  t_crit_low <- qt(alpha/2, df)
  t_crit_high <- qt(1 - alpha/2, df)

  # Conclusión
  resultado <- if (t_calc < t_crit_low | t_calc > t_crit_high) {
    "Reject"
  } else {
    "Fail to Reject"
  }

  # Retornar lista
  return(list(
    t_calc = t_calc,
    t_crit_low = t_crit_low,
    t_crit_high = t_crit_high,
    resultado = resultado
  ))
}


resultado <- prueba_t(xbar = 27.5, mu0 = 30, s = 5, n = 16)
print(resultado)

codigo_funcion <- paste(deparse(prueba_t), collapse = "\n")

In [ ]:
#@title Calificador Ejercicio 2
library(httr)
library(jsonlite)

# Datos del estudiante
codigo_estudiante <- "" #@param {type:"string"}
nombre_ejercicio <- "Prueba t una muestra"

# === Preparar JSON para enviar al backend ===
payload <- list(
  codigo_estudiante = codigo_estudiante,
  t_calc = resultado$t_calc,
  t_crit_low = resultado$t_crit_low,
  t_crit_high = resultado$t_crit_high,
  resultado = resultado$resultado,
  codigo_funcion = codigo_funcion
)

# === Enviar al backend ===
res <- POST(
  "https://147.185.221.25:28151/api/pract/evaluar-t-una-muestra",
  body = payload,
  encode = "json",
  config = config(
    ssl_verifypeer = TRUE,
    ssl_verifyhost = 2,
    cainfo = "fullchain.pem"
  )
)

# === Parsear respuesta JSON ===
respuesta <- content(res, as = "parsed", encoding = "UTF-8")

# === Mostrar resultados de forma elegante ===
cat("===== Resultado del ejercicio =====\n\n")

if (!is.null(respuesta$error)) {
  if (length(respuesta$error) > 1) {
    cat("Se encontraron los siguientes errores:\n")
    for (err in respuesta$error) {
      cat("-", err, "\n")
    }
  } else {
    cat("Error:", respuesta$error, "\n")
  }
} else {
  cat("Mensaje del servidor:", respuesta$mensaje, "\n\n")

  if (!is.null(respuesta$fallos) && length(respuesta$fallos) > 0) {
    cat("Se encontraron fallos en los siguientes items:\n")
    for (f in respuesta$fallos) {
      cat("-", f, "\n")
    }
  } else {
    cat("¡Todos los resultados son correctos! ✅\n")
  }
}

# **Ejercicio 3**

Una población normal con varianza desconocida tiene una media de $20$. ¿Es posible obtener una muestra aleatoria de tamaño $9$ de esta población con una media de $24$ y una desviación estándar de $4.1$? Si no fuera posible, ¿a qué conclusión llegaría?

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.


In [ ]:
test_t <- function(xbar, mu0, s, n, alpha = 0.05) {
  # Estadístico t
  t_calc <- (xbar - mu0) / (s / sqrt(n))

  # Grados de libertad
  df <- n - 1

  # Valores críticos (prueba bilateral)
  t_crit_low <- qt(alpha/2, df)
  t_crit_high <- qt(1 - alpha/2, df)

  # Decisión
  resultado <- if (t_calc < t_crit_low | t_calc > t_crit_high) {
    "Reject"
  } else {
    "Fail to Reject"
  }

  return(list(
    t_calc = t_calc,
    t_crit_low = t_crit_low,
    t_crit_high = t_crit_high,
    resultado = resultado
  ))
}

resultado <- test_t(xbar = 24, mu0 = 20, s = 4.1, n = 9, alpha = 0.05)
print(resultado)

codigo_funcion <- paste(deparse(test_t), collapse = "\n")

print(codigo_funcion)

In [ ]:
#@title Calificador Ejercicio 3 - Práctica 2
library(httr)
library(jsonlite)

# Datos del estudiante
codigo_estudiante <- "" #@param {type:"string"}
nombre_ejercicio <- "Ejercicio 3 - Práctica 2"

# === Preparar JSON para enviar al backend ===
payload <- list(
  codigo_estudiante = codigo_estudiante,
  t_calc = resultado$t_calc,
  t_crit_low = resultado$t_crit_low,
  t_crit_high = resultado$t_crit_high,
  resultado = resultado$resultado,
  codigo_funcion = codigo_funcion
)

# === Enviar al backend ===
res <- POST(
  "https://147.185.221.25:28151/api/pract/ejer3_Prac2",  # Nuevo endpoint
  body = payload,
  encode = "json",
  config = config(
    ssl_verifypeer = TRUE,
    ssl_verifyhost = 2,
    cainfo = "fullchain.pem"
  )
)

# === Parsear respuesta JSON ===
respuesta <- content(res, as = "parsed", encoding = "UTF-8")

# === Mostrar resultados de forma elegante ===
cat("===== Resultado del ejercicio =====\n\n")

if (!is.null(respuesta$error)) {
  if (length(respuesta$error) > 1) {
    cat("Se encontraron los siguientes errores:\n")
    for (err in respuesta$error) {
      cat("-", err, "\n")
    }
  } else {
    cat("Error:", respuesta$error, "\n")
  }
} else {
  cat("Mensaje del servidor:", respuesta$mensaje, "\n\n")

  if (!is.null(respuesta$fallos) && length(respuesta$fallos) > 0) {
    cat("Se encontraron fallos en los siguientes items:\n")
    for (f in respuesta$fallos) {
      cat("-", f, "\n")
    }
  } else {
    cat("¡Todos los resultados son correctos! ✅\n")
  }
}


# **Ejercicio 4**

Un fabricante de cierta marca de barras de cereal con bajo contenido de grasa afirma que el contenido promedio de grasa saturada en éstas, es de 0.5 gramos. En una muestra aleatoria de 8 barras de cereal de esta marca se encontró que su contenido de grasa saturada era de:

$$0.6, 0.7, 0.7, 0.3, 0.4, 0.5, 0.4, 0.2$$

¿Estaría de acuerdo con tal afirmación? Suponga una distribución normal.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [ ]:
custom_t_test <- function(x, mu0, alpha = 0.05) {
  n <- length(x)
  xbar <- mean(x)
  s <- sd(x)

  # Estadístico t
  t_calc <- (xbar - mu0) / (s / sqrt(n))

  # Grados de libertad
  df <- n - 1

  # Valores críticos (bilateral)
  t_crit_low <- qt(alpha/2, df)
  t_crit_high <- qt(1 - alpha/2, df)

  # Decisión
  resultado <- if (t_calc < t_crit_low | t_calc > t_crit_high) {
    "Reject"
  } else {
    "Fail to Reject"
  }

  return(list(
    t_calc = t_calc,
    t_crit_low = t_crit_low,
    t_crit_high = t_crit_high,
    resultado = resultado
  ))
}

x <- c(0.6,0.7,0.7,0.3,0.4,0.5,0.4,0.2)
resultado <- custom_t_test(x, mu0 = 0.5, alpha = 0.05)
print(resultado)

codigo_funcion <- paste(deparse(custom_t_test), collapse = "\n")

In [ ]:
#@title Calificador Ejercicio 4 - Práctica 2
library(httr)
library(jsonlite)

# Datos del estudiante
codigo_estudiante <- "" #@param {type:"string"}
nombre_ejercicio <- "Ejercicio 4 - Práctica 2"

# === Preparar JSON para enviar al backend ===
payload <- list(
  codigo_estudiante = codigo_estudiante,
  t_calc = resultado$t_calc,
  t_crit_low = resultado$t_crit_low,
  t_crit_high = resultado$t_crit_high,
  resultado = resultado$resultado,
  codigo_funcion = codigo_funcion
)

# === Enviar al backend ===
res <- POST(
  "https://147.185.221.25:28151/api/pract/ejer4_Prac2",  # Nuevo endpoint
  body = payload,
  encode = "json",
  config = config(
    ssl_verifypeer = TRUE,
    ssl_verifyhost = 2,
    cainfo = "fullchain.pem"
  )
)

# === Parsear respuesta JSON ===
respuesta <- content(res, as = "parsed", encoding = "UTF-8")

# === Mostrar resultados de forma elegante ===
cat("===== Resultado del ejercicio =====\n\n")

if (!is.null(respuesta$error)) {
  if (length(respuesta$error) > 1) {
    cat("Se encontraron los siguientes errores:\n")
    for (err in respuesta$error) {
      cat("-", err, "\n")
    }
  } else {
    cat("Error:", respuesta$error, "\n")
  }
} else {
  cat("Mensaje del servidor:", respuesta$mensaje, "\n\n")

  if (!is.null(respuesta$fallos) && length(respuesta$fallos) > 0) {
    cat("Se encontraron fallos en los siguientes items:\n")
    for (f in respuesta$fallos) {
      cat("-", f, "\n")
    }
  } else {
    cat("¡Todos los resultados son correctos! ✅\n")
  }
}
